<a href="https://colab.research.google.com/github/rubuntu/uaa-417-sistemas-de-gestion-de-bases-de-datos-avanzados/blob/main/10_Data_Munging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
import pandas as pd
import json
import re

matplotlib.rcParams['savefig.dpi'] = 144

# Manipulación de datos

## Datos relacionales

El tipo de datos más simple que hemos visto puede consistir en una sola tabla con algunas columnas y algunas filas. Este tipo de datos es fácil de analizar y calcular y, por lo general, queremos reducir nuestros datos a una sola tabla antes de comenzar a ejecutar algoritmos de aprendizaje automático. Sin embargo, los datos del mundo real no encajan necesariamente en este paradigma. La mayoría de los datos del mundo real son desordenados y complicados, lo que no encaja bien en un formato tabular y tendremos que hacer algo de trabajo para reducir esta complejidad. Además, en muchos casos podemos reducir nuestro costo de memoria al no mantener los datos en una sola tabla, sino en un conjunto de estructuras de datos con relaciones definidas entre ellos.

Aquí exploraremos un poco de datos y veremos cómo la combinación de diferentes conjuntos de datos puede ayudarnos a generar características útiles.

Primero necesitamos algunos datos. Haremos uso de algunos datos de Wikipedia y usaremos la función `read_html` de pandas para extraer los datos de una página web en particular. Estudiaremos las 10 principales empresas de Fortune Global 500 que convenientemente tienen su propia página de Wikipedia.

Descargaremos los datos en forma de tabla, pero trabajaremos con ellos como una lista de diccionarios, esto nos permitirá acostumbrarnos a trabajar con datos no estructurados.

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/Fortune_Global_500', header=0)[0]
fortune_500 = json.loads(df.to_json(orient="records"))
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$612.3 billion
1,2,Saudi Aramco,Saudi Arabia,Energy,$603.7 billion
2,3,State Grid,China,Energy,$530.0 billion
3,4,Amazon,United States,Internet services and retailing,$514.0 billion
4,5,China National Petroleum,China,Petroleum,$483.1 billion
5,6,Sinopec Group,China,Petroleum,$471.2 billion
6,7,ExxonMobil,United States,Petroleum,$413.7 billion
7,8,Apple,United States,Technology,$394.3 billion
8,9,Shell,United Kingdom,Petroleum,$386.2 billion
9,10,UnitedHealth Group,United States,Health care,$324.2 billion


Veamos la data.

In [ ]:
fortune_500

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$612.3 billion'},
 {'Rank': 2,
  'Company': 'Saudi Aramco',
  'Country': 'Saudi Arabia',
  'Industry': 'Energy',
  'Revenue in USD': '$603.7 billion'},
 {'Rank': 3,
  'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Energy',
  'Revenue in USD': '$530.0 billion'},
 {'Rank': 4,
  'Company': 'Amazon',
  'Country': 'United States',
  'Industry': 'Internet services and retailing',
  'Revenue in USD': '$514.0 billion'},
 {'Rank': 5,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$483.1 billion'},
 {'Rank': 6,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$471.2 billion'},
 {'Rank': 7,
  'Company': 'ExxonMobil',
  'Country': 'United States',
  'Industry': 'Petroleum',
  'Revenue in USD': '$413.7 billion'},
 {'Rank': 8,
  'Company': 'Apple',
  'Country': 

Este es un buen comienzo para nuestro análisis, sin embargo, en realidad no hay mucha información aquí, necesitaremos incorporar fuentes de datos adicionales para comprender mejor a estas empresas.

La primera pregunta que podríamos hacernos es cuántos empleados se necesitan para obtener esos ingresos, en otras palabras, ¿cuál es el ingreso por empleado? Afortunadamente, también podemos usar Wikipedia para obtener esos datos; hemos extraído estos datos manualmente (todos de Wikipedia) y hemos creado el siguiente diccionario.

In [ ]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999}
]

Algunos datos tienen un nombre ligeramente diferente al de nuestro conjunto original, por lo que mantendremos un diccionario de asignaciones entre ambos. Tenga en cuenta que solo incluimos la asignación en el diccionario si existe una diferencia.

In [ ]:
mapping = {"China Petrochemical Corporation":"Sinopec Group",
           "State Grid Corporation of China":"State Grid",
           "China National Petroleum Corporation":"China National Petroleum",
           "Toyota Motor Corporation": "Toyota Motor",
           "Volkswagen AG":"Volkswagen",
           "Berkshire Hathaway Inc.":"Berkshire Hathaway",
           "Apple Inc.": "Apple",
           "Exxon Mobile Corporation": "Exxon Mobil"
          }

Estos datos son uno a uno, lo que significa que los datos contenidos en una fuente solo se alinean con un único elemento en la otra fuente, por lo que deberíamos poder unirlos. Sin embargo, sabemos que los datos no están en un buen estado para unirlos en este momento. Esto se debe a dos razones:

1. No se alinearán todos los nombres (necesitamos usar nuestro mapeo)
2. La estructura de "lista" no está optimizada para buscar elementos.

Si bien para 10 elementos la segunda razón realmente no importará, para conjuntos de datos más grandes, estas consideraciones de rendimiento son extremadamente importantes. Podemos convertir esta lista de diccionarios en un diccionario de diccionarios, de modo que podamos acceder rápidamente a cada elemento de los datos.

In [ ]:
dict_data = {k["name"] : k for k in other_data}
dict_data

{'Walmart': {'name': 'Walmart', 'employees': 2300000, 'year founded': 1962},
 'State Grid Corporation of China': {'name': 'State Grid Corporation of China',
  'employees': 927839,
  'year founded': 2002},
 'China Petrochemical Corporation': {'name': 'China Petrochemical Corporation',
  'employees': 358571,
  'year founded': 1998},
 'China National Petroleum Corporation': {'name': 'China National Petroleum Corporation',
  'employees': 1636532,
  'year founded': 1988},
 'Toyota Motor Corporation': {'name': 'Toyota Motor Corporation',
  'employees': 364445,
  'year founded': 1937},
 'Volkswagen AG': {'name': 'Volkswagen AG',
  'employees': 642292,
  'year founded': 1937},
 'Royal Dutch Shell': {'name': 'Royal Dutch Shell',
  'employees': 92000,
  'year founded': 1907},
 'Berkshire Hathaway Inc.': {'name': 'Berkshire Hathaway Inc.',
  'employees': 377000,
  'year founded': 1839},
 'Apple Inc.': {'name': 'Apple Inc.',
  'employees': 123000,
  'year founded': 1976},
 'Exxon Mobile Corporatio

**Pregunta:** Si tuviéramos muchas entradas en `other_data`, podríamos mostrar una pequeña parte imprimiendo `other_data[:5]`. Con los marcos de datos podríamos usar `df.head()`. ¿Puedes pensar en una forma de imprimir una pequeña parte de un diccionario?

Ahora podemos calcular fácilmente los ingresos por empleado. Necesitamos mapear el valor de "Compañía" en nuestros datos originales con la columna "nombre" de estos otros datos, pero también necesitamos usar el mapeo para asegurarnos de que las columnas se alineen. En general, no queremos modificar nuestros datos originales, así que hagamos una nueva lista de diccionarios con esta nueva característica (ingresos por empleado). Mientras hacemos esto, necesitaremos manejar la conversión de algunos números como "$500 mil millones" a un valor numérico. Vamos a crear una función para hacer esto.

In [ ]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

Ahora podemos convertir este ingreso, pero desafortunadamente el mapeo que hemos escrito va en la dirección incorrecta, invirtámoslo.

In [ ]:
inv_map = {v: k for k, v in mapping.items()}
inv_map

{'Sinopec Group': 'China Petrochemical Corporation',
 'State Grid': 'State Grid Corporation of China',
 'China National Petroleum': 'China National Petroleum Corporation',
 'Toyota Motor': 'Toyota Motor Corporation',
 'Volkswagen': 'Volkswagen AG',
 'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
 'Apple': 'Apple Inc.',
 'Exxon Mobil': 'Exxon Mobile Corporation'}

**Pregunta:** ¿Qué pasaría si el «mapeo» no fuera uno a uno, es decir, si varias claves tuvieran el mismo valor?

Ahora deberíamos poder crear algunas funciones para calcular estos ingresos por empleado y crear una lista de datos.

In [ ]:
def rev_per_emp(company):
    name = company['Company']
    # Intentamos obtener el número de empleados, si no lo encontramos, usamos un valor por defecto (puede ser 1 o cualquier otro).
    employee_data = dict_data.get(inv_map.get(name, name), {})
    n_employees = employee_data.get('employees')

    if n_employees is None:
        print(f"Advertencia: No se encontraron empleados para la empresa '{name}'. Asignando valor predeterminado de 1.")
        n_employees = 1  # O cualquier valor por defecto que prefieras

    company['rev per emp'] = convert_revenue(company['Revenue in USD']) / n_employees
    return company

def compute_copy(d, func):
    return func({k: v for k, v in d.items()})

# En Python 3, map devuelve un iterador, por lo que puedes convertirlo en una lista
data = list(map(lambda x: compute_copy(x, rev_per_emp), fortune_500))


Advertencia: No se encontraron empleados para la empresa 'Saudi Aramco'. Asignando valor predeterminado de 1.
Advertencia: No se encontraron empleados para la empresa 'Amazon'. Asignando valor predeterminado de 1.
Advertencia: No se encontraron empleados para la empresa 'ExxonMobil'. Asignando valor predeterminado de 1.
Advertencia: No se encontraron empleados para la empresa 'Shell'. Asignando valor predeterminado de 1.
Advertencia: No se encontraron empleados para la empresa 'UnitedHealth Group'. Asignando valor predeterminado de 1.


Echemos un vistazo a nuestros datos nuevos y también a los antiguos para asegurarnos de que no mutamos nada.

In [ ]:
data[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$612.3 billion',
  'rev per emp': 266217.39130434784},
 {'Rank': 2,
  'Company': 'Saudi Aramco',
  'Country': 'Saudi Arabia',
  'Industry': 'Energy',
  'Revenue in USD': '$603.7 billion',
  'rev per emp': 603700000000.0}]

In [ ]:
fortune_500[:2]

[{'Rank': 1,
  'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Revenue in USD': '$612.3 billion'},
 {'Rank': 2,
  'Company': 'Saudi Aramco',
  'Country': 'Saudi Arabia',
  'Industry': 'Energy',
  'Revenue in USD': '$603.7 billion'}]

Ahora podemos ordenar estos valores. Primero podemos seleccionar los elementos que nos interesan y luego ordenar esa lista.

In [ ]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data],
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

[('Saudi Aramco', 603700000000.0),
 ('Amazon', 514000000000.0),
 ('ExxonMobil', 413700000000.0),
 ('Shell', 386200000000.0),
 ('UnitedHealth Group', 324200000000.0),
 ('Apple', 3205691.0569105693),
 ('Sinopec Group', 1314105.1563009836),
 ('State Grid', 571219.7913646656),
 ('China National Petroleum', 295197.4052447493),
 ('Walmart', 266217.39130434784)]

Esto da como resultado un orden muy diferente. ¿Qué nos dice esto sobre las empresas?

Ahora, incorporemos otros datos (esto es ciencia de datos, ¡siempre más datos es mejor!). Podemos ver que estas empresas pertenecen a diferentes industrias, descubramos cuáles son.

In [ ]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

Counter({'Retail': 1,
         'Energy': 2,
         'Internet services and retailing': 1,
         'Petroleum': 4,
         'Technology': 1,
         'Health care': 1})

Una cosa que nos interesaría saber es qué tipo de participación de mercado tienen en la industria específica a la que pertenecen. Veamos las dos industrias que categorizan a 6 de las 10 principales, "Automóviles" y "Petróleo". Podemos seleccionar solo esos elementos de nuestros datos para trabajar con ellos.

In [ ]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data

[{'Rank': 5,
  'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$483.1 billion',
  'rev per emp': 295197.4052447493},
 {'Rank': 6,
  'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Revenue in USD': '$471.2 billion',
  'rev per emp': 1314105.1563009836},
 {'Rank': 7,
  'Company': 'ExxonMobil',
  'Country': 'United States',
  'Industry': 'Petroleum',
  'Revenue in USD': '$413.7 billion',
  'rev per emp': 413700000000.0},
 {'Rank': 9,
  'Company': 'Shell',
  'Country': 'United Kingdom',
  'Industry': 'Petroleum',
  'Revenue in USD': '$386.2 billion',
  'rev per emp': 386200000000.0}]

Puede darse el caso de que cada categoría en particular tenga una métrica relevante diferente para la participación de mercado. Por ejemplo, podríamos analizar los ingresos totales de una empresa automotriz o los automóviles producidos.

Por lo tanto, para la industria automotriz, analizaremos el porcentaje total de automóviles producidos. Podemos obtener estos datos nuevamente de Wikipedia.

In [ ]:
df_list = pd.read_html("https://en.wikipedia.org/wiki/Automotive_industry", header=0)
car_totals = json.loads(df_list[1].to_json(orient="records"))
car_by_man = json.loads(df_list[3].to_json(orient='records'))

In [ ]:
df_list[1]

,Year,Production,Change,Source
0,1997,54434000,—,[27]
1,1998,52987000,2.7%,[27]
2,1999,56258892,6.2%,[28]
3,2000,58374162,3.8%,[29]
4,2001,56304925,3.5%,[30]
5,2002,58994318,4.8%,[31]
6,2003,60663225,2.8%,[32]
7,2004,64496220,6.3%,[33]
8,2005,66482439,3.1%,[34]
9,2006,69222975,4.1%,[35]


In [ ]:
df_list[2]

,Country,Produced vehicles 2023[54]
0,China (plus Taiwan),"30,160,966 (30,446,928)"
1,USA,10611555
2,Japan,8997440
3,India,5851507
4,Republic of Korea,4243597
5,Germany,4109371
6,Mexico,4002047
7,Spain,2451221
8,Brazil,2324838
9,Thailand,1841663


In [ ]:
df_list[3]

,Rank[a],Group,Country,Produced vehicles (2017)[55],Sold vehicles (2018),Sold vehicles (2019)[57]
0,1,Toyota,Japan,10466051,10521134,10741556
1,2,Volkswagen Group,Germany,10382334,10831232,10975352
2,3,General Motors (except SAIC-GM-Wuling)[b],United States,"9,027,658 (6,856,880)",8787233,7724163
3,4,Hyundai,South Korea,7218391,7437209,7189893
4,5,Ford,United States,6386818,5734217,5385972
5,6,Nissan,Japan,5769277,5653743,5176211
6,7,Honda,Japan,5235842,5265892,5323319
7,8,Fiat-Chrysler (now part of Stellantis),Italy / United States,4600847,4841366,4612673
8,9,Renault,France,4153589,3883987,3749815
9,10,PSA Group (now part of Stellantis),France,3649742,4126349,3479152


In [ ]:
df_list[4]

,Rank[c],Group,Country,Produced vehicles (2013)[58],Produced vehicles (2012)[59],Produced vehicles (2011)[60]
0,1.0,Toyota,Japan,10324995,10104424,8050181
1,2.0,General Motors,United States,9628912,9285425,9031670
2,3.0,Volkswagen Group,Germany,9379229,9254742,8525573
3,4.0,Hyundai,South Korea,7233080,7126413,6616858
4,5.0,Ford,United States,6077126,5595483,5516931
5,6.0,Nissan,Japan,4950924,4889379,4631673
6,7.0,Fiat / FCA,Italy,4681704,4 498 722[d],2336954
7,8.0,Honda,Japan,4298390,4110857,2909016
8,9.0,PSA Peugeot Citroën,France,2833781,2911764,3582410
9,10.0,Suzuki,Japan,2842133,2893602,2725899


In [ ]:
car_totals[:2]

[{'Year': 1997, 'Production': 54434000, 'Change': '—', 'Source': '[27]'},
 {'Year': 1998, 'Production': 52987000, 'Change': '2.7%', 'Source': '[27]'}]

In [ ]:
car_by_man[:2]

[{'Rank[a]': 1,
  'Group': 'Toyota',
  'Country': 'Japan',
  'Produced vehicles (2017)[55]': '10466051',
  'Sold vehicles (2018)': 10521134,
  'Sold vehicles (2019)[57]': 10741556},
 {'Rank[a]': 2,
  'Group': 'Volkswagen Group',
  'Country': 'Germany',
  'Produced vehicles (2017)[55]': '10382334',
  'Sold vehicles (2018)': 10831232,
  'Sold vehicles (2019)[57]': 10975352}]

Ahora tomemos solo los grupos que nos interesan y dividámoslos por la producción total que tomaremos como el último año.

In [ ]:
total_prod = sorted((i["Year"], i['Production']) for i in car_totals)[-1][1]
total_prod


85016728

Ahora podemos encontrar la cuota de mercado de cada una de las empresas automovilísticas. Para ello, utilizaremos un diccionario de cuotas de mercado. Nuevamente, tendremos que hacer un seguimiento de algunas pequeñas diferencias de nombres.

In [ ]:
# Convertimos los valores de vehículos a enteros, eliminando caracteres no deseados
car_by_man_dict = {
    i['Group']: int(re.findall(r'\d+', i['Produced vehicles (2017)[55]'].replace(',', ''))[0])
    for i in car_by_man
}

market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name] / float(total_prod)

market_share

{'Toyota': 0.12310578454630718, 'Volkswagen': 0.12212107245529374}

Ahora podemos hacer lo mismo para la industria petrolera, pero en este caso, calculemos la participación de mercado por ingresos. En Wikipedia, podemos encontrar una lista de compañías petroleras por ingresos. Si bien no es una lista completa, tiene suficientes compañías como para que no esperemos que las compañías que no figuran en la lista contribuyan en gran medida a nuestro análisis.

In [ ]:
# Cargar la tabla desde Wikipedia
rev = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_oil_and_gas_companies_by_revenue", header=1)[0]

# Filtrar las columnas que te interesan
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']

# Limpiar la columna 'Revenue' extrayendo solo números (tanto enteros como decimales)
rev['Revenue'] = rev['Revenue'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x))[0] if re.findall(r'\d+\.\d+|\d+', str(x)) else '0')

# Convertir la columna 'Revenue' a tipo float
rev['Revenue'] = rev['Revenue'].astype(float)

# Convertir el DataFrame a una lista de diccionarios
oil_data = json.loads(rev.to_json(orient="records"))
print(oil_data[:2])


[{'Company': 'Sonangol Group', 'Revenue': 17.6}, {'Company': 'YPF', 'Revenue': 15.3}]


Ahora podemos calcular los totales y la cuota de mercado. Dado que los datos que aparecen aquí pueden ser ligeramente diferentes (quizás más antiguos) que los datos originales, calcularemos la cuota de mercado de cada empresa dentro de este conjunto de datos y luego extraeremos los números que nos interesan.

In [ ]:
# Calcular el total de revenue
total = sum([float(i['Revenue']) for i in oil_data])

# Calcular la participación de mercado de cada compañía
shares = {i['Company']: float(i['Revenue']) / total for i in oil_data}

# Imprimir el total
print(f"Total Revenue: {total}")
print(f"Market Shares: {shares}")


Total Revenue: 4930.200000000001
Market Shares: {'Sonangol Group': 0.0035698348951361, 'YPF': 0.003103322380430814, 'APA Group': 0.00038537990345219254, 'Origin Energy': 0.002150014198206969, 'Santos': 0.0006287777372114721, 'Woodside Energy': 0.0007910429597176584, 'OMV Group': 0.0046245588414263105, 'SOCAR': 0.01091233621354103, 'nogaholding': 0.0012169891687963976, 'Bangladesh Petroleum Corporation': 0.0, 'Petrobangla': 0.00010141576406636645, 'Belneftekhim': 0.002575960407285708, 'YPFB': 0.0011358565575433043, 'Petrobras': 0.01578029288872662, 'Ultrapar': 0.0050302218976917765, 'Brunei Energy Services and Trading': 0.00036509675063891927, 'Bulgarian Energy Holding': 0.0006490608900247453, 'AltaGas': 0.00038537990345219254, 'ARC Resources': 0.00016226522250618633, 'Brookfield Infrastructure Partners': 0.0, 'Canadian Natural Resources': 0.0028599245466715342, 'Crescent Point Energy': 0.0005070788203318323, 'Cenovus Energy': 0.002697659324165348, 'Enbridge': 0.006936838262139466, 'Ene

In [ ]:
import pandas as pd
import json
import re

# Cargar la tabla desde Wikipedia
rev = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_oil_and_gas_companies_by_revenue", header=1)[0]

# Filtrar las columnas que te interesan
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']

# Limpiar la columna 'Revenue' extrayendo solo números (tanto enteros como decimales)
rev['Revenue'] = rev['Revenue'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x))[0] if re.findall(r'\d+\.\d+|\d+', str(x)) else '0')

# Convertir la columna 'Revenue' a tipo float
rev['Revenue'] = rev['Revenue'].astype(float)

# Convertir el DataFrame a una lista de diccionarios
oil_data = json.loads(rev.to_json(orient="records"))

# Calcular el total de revenue
total = sum([float(i['Revenue']) for i in oil_data])

# Calcular la participación de mercado de cada compañía
shares = {i['Company']: float(i['Revenue']) / total for i in oil_data}

# Imprimir el total
print(f"Total Revenue: {total}")
print(f"Market Shares: {shares}")


Total Revenue: 4930.200000000001
Market Shares: {'Sonangol Group': 0.0035698348951361, 'YPF': 0.003103322380430814, 'APA Group': 0.00038537990345219254, 'Origin Energy': 0.002150014198206969, 'Santos': 0.0006287777372114721, 'Woodside Energy': 0.0007910429597176584, 'OMV Group': 0.0046245588414263105, 'SOCAR': 0.01091233621354103, 'nogaholding': 0.0012169891687963976, 'Bangladesh Petroleum Corporation': 0.0, 'Petrobangla': 0.00010141576406636645, 'Belneftekhim': 0.002575960407285708, 'YPFB': 0.0011358565575433043, 'Petrobras': 0.01578029288872662, 'Ultrapar': 0.0050302218976917765, 'Brunei Energy Services and Trading': 0.00036509675063891927, 'Bulgarian Energy Holding': 0.0006490608900247453, 'AltaGas': 0.00038537990345219254, 'ARC Resources': 0.00016226522250618633, 'Brookfield Infrastructure Partners': 0.0, 'Canadian Natural Resources': 0.0028599245466715342, 'Crescent Point Energy': 0.0005070788203318323, 'Cenovus Energy': 0.002697659324165348, 'Enbridge': 0.006936838262139466, 'Ene

Ahora podemos retirar las empresas que nos importan en la industria petrolera.

In [ ]:
petro_companies = [i['Company'] for i in data if i['Industry'] == 'Petroleum']
petro_companies

['China National Petroleum', 'Sinopec Group', 'ExxonMobil', 'Shell']

Lets check if these are all in the our shares dictionary.

In [ ]:
[(i, i in shares) for i in petro_companies]

[('China National Petroleum', False),
 ('Sinopec Group', False),
 ('ExxonMobil', True),
 ('Shell', True)]

Algunas de estas empresas están directamente allí, y buscando en nuestro diccionario podemos ver que las demás están allí sin nombres exactos.

In [ ]:
shares.keys()

dict_keys(['Sonangol Group', 'YPF', 'APA Group', 'Origin Energy', 'Santos', 'Woodside Energy', 'OMV Group', 'SOCAR', 'nogaholding', 'Bangladesh Petroleum Corporation', 'Petrobangla', 'Belneftekhim', 'YPFB', 'Petrobras', 'Ultrapar', 'Brunei Energy Services and Trading', 'Bulgarian Energy Holding', 'AltaGas', 'ARC Resources', 'Brookfield Infrastructure Partners', 'Canadian Natural Resources', 'Crescent Point Energy', 'Cenovus Energy', 'Enbridge', 'Enerplus', 'Frontera Energy', 'Gibson Energy', 'Inter Pipeline', 'Keyera', 'MEG Energy', 'Paramount Resources', 'Pembina Pipeline', 'Suncor Energy', 'Tourmaline Oil', 'TC Energy', 'Vermilion Energy', 'Whitecap Resources', 'Empresas Copec', 'Empresa Nacional del Petróleo', 'China National Offshore Oil', 'China Suntien Green Energy', 'COSCO Shipping Energy', 'China National Petroleum Corporation', 'Sinopec (China Petrochemical)', 'Guanghui Energy', 'Ecopetrol', 'INA d.d.', 'Moravské naftové doly (MND)', 'Petroecuador', 'Neste', 'Engie', 'Schlumbe

Entonces, hagamos una coincidencia aproximada. Esta será bastante simple, donde intentará hacer coincidir palabras en un nombre y tomar el número máximo de coincidencias.

In [ ]:
def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.items()]
    return max(overlaps, key=lambda x : x[1])[0]

In [ ]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print("matched {} to {}".format(i, match))
    market_share[i] = shares[match]

matched China National Petroleum to China National Petroleum Corporation
matched Sinopec Group to Sonangol Group
matched ExxonMobil to ExxonMobil
matched Shell to Shell


In [ ]:
market_share

{'Toyota': 0.12310578454630718,
 'Volkswagen': 0.12212107245529374,
 'China National Petroleum': 0.07024055819236541,
 'Sinopec Group': 0.0035698348951361,
 'ExxonMobil': 0.04955174232282666,
 'Shell': 0.042959717658512836}

## Por industria
Tenemos algunos buenos ejemplos de manipulación de datos, ahora veamos un ejemplo de cómo mantener los datos de manera relacional. Supongamos que queremos agregar otra característica, que es el crecimiento de cada industria. Si tuviéramos que almacenar estos datos como una sola cantidad, estaríamos ahorrando un montón de información adicional. Sería mucho mejor extraer esta información y mantenerla en una sola tabla para no tener que replicarla por industria.

## Con Pandas

Ahora también podemos realizar estos mismos cálculos con Pandas. Veamos cómo se comparan.

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/Fortune_Global_500', header=0)[0]
print(df)

   Rank                   Company         Country  \
0     1                   Walmart   United States   
1     2              Saudi Aramco    Saudi Arabia   
2     3                State Grid           China   
3     4                    Amazon   United States   
4     5  China National Petroleum           China   
5     6             Sinopec Group           China   
6     7                ExxonMobil   United States   
7     8                     Apple   United States   
8     9                     Shell  United Kingdom   
9    10        UnitedHealth Group   United States   

                          Industry  Revenue in USD  
0                           Retail  $612.3 billion  
1                           Energy  $603.7 billion  
2                           Energy  $530.0 billion  
3  Internet services and retailing  $514.0 billion  
4                        Petroleum  $483.1 billion  
5                        Petroleum  $471.2 billion  
6                        Petroleum  $413.7 bi

In [ ]:
# Datos de empleados y empresas
employees_data = {
    "Company": [
        "Walmart",
        "Saudi Aramco",
        "State Grid",
        "Amazon",
        "China National Petroleum",
        "Sinopec",
        "ExxonMobil",
        "Apple",
        "Shell",
        "UnitedHealth Group"
    ],
    "Employees": [
        2100000,
        68500,
        1300000,
        1540000,
        1500000,
        385000,
        62000,
        164000,
        93000,
        400000
    ]
}

# Crear DataFrame con la información de empleados
df_employees = pd.DataFrame(employees_data)

# Cargar la tabla desde Wikipedia
df_fortune = pd.read_html('https://en.wikipedia.org/wiki/Fortune_Global_500', header=0)[0]

# Unir ambas tablas por la columna 'Company'
df = pd.merge(df_fortune, df_employees, how='left', on='Company')

# Mostrar el resultado con la nueva columna de empleados
df


,Rank,Company,Country,Industry,Revenue in USD,Employees
0,1,Walmart,United States,Retail,$612.3 billion,2100000.0
1,2,Saudi Aramco,Saudi Arabia,Energy,$603.7 billion,68500.0
2,3,State Grid,China,Energy,$530.0 billion,1300000.0
3,4,Amazon,United States,Internet services and retailing,$514.0 billion,1540000.0
4,5,China National Petroleum,China,Petroleum,$483.1 billion,1500000.0
5,6,Sinopec Group,China,Petroleum,$471.2 billion,NaN
6,7,ExxonMobil,United States,Petroleum,$413.7 billion,62000.0
7,8,Apple,United States,Technology,$394.3 billion,164000.0
8,9,Shell,United Kingdom,Petroleum,$386.2 billion,93000.0
9,10,UnitedHealth Group,United States,Health care,$324.2 billion,400000.0


In [ ]:
# Función para convertir la columna 'Revenue in USD' a un número manejable
df['rev'] = df['Revenue in USD'].apply(convert_revenue)

# Función más segura para obtener el número de empleados
df['employees'] = df['Company'].apply(lambda x: dict_data.get(inv_map.get(x, x), {}).get('employees', 0))

# Asegurarse de que no haya divisiones por cero en 'employees' y convertir empleados a float
df['rev_per_employee'] = df.apply(lambda row: row['rev'] / float(row['employees']) if row['employees'] > 0 else None, axis=1)

# Ordenar por 'rev_per_employee' de forma descendente
df.sort_values(by='rev_per_employee', ascending=False)


,Rank,Company,Country,Industry,Revenue in USD,Employees,rev,employees,rev_per_employee
7,8,Apple,United States,Technology,$394.3 billion,164000.0,3.943000e+11,123000,3.205691e+06
5,6,Sinopec Group,China,Petroleum,$471.2 billion,NaN,4.712000e+11,358571,1.314105e+06
2,3,State Grid,China,Energy,$530.0 billion,1300000.0,5.300000e+11,927839,5.712198e+05
4,5,China National Petroleum,China,Petroleum,$483.1 billion,1500000.0,4.831000e+11,1636532,2.951974e+05
0,1,Walmart,United States,Retail,$612.3 billion,2100000.0,6.123000e+11,2300000,2.662174e+05
1,2,Saudi Aramco,Saudi Arabia,Energy,$603.7 billion,68500.0,6.037000e+11,0,NaN
3,4,Amazon,United States,Internet services and retailing,$514.0 billion,1540000.0,5.140000e+11,0,NaN
6,7,ExxonMobil,United States,Petroleum,$413.7 billion,62000.0,4.137000e+11,0,NaN
8,9,Shell,United Kingdom,Petroleum,$386.2 billion,93000.0,3.862000e+11,0,NaN
9,10,UnitedHealth Group,United States,Health care,$324.2 billion,400000.0,3.242000e+11,0,NaN


In [ ]:
df_list = pd.read_html("https://en.wikipedia.org/wiki/Automotive_industry", header=0)
df_totals = df_list[1]
df_by_man = df_list[3]

In [ ]:
df_totals

,Year,Production,Change,Source
0,1997,54434000,—,[27]
1,1998,52987000,2.7%,[27]
2,1999,56258892,6.2%,[28]
3,2000,58374162,3.8%,[29]
4,2001,56304925,3.5%,[30]
5,2002,58994318,4.8%,[31]
6,2003,60663225,2.8%,[32]
7,2004,64496220,6.3%,[33]
8,2005,66482439,3.1%,[34]
9,2006,69222975,4.1%,[35]


In [ ]:
df_by_man

,Rank[a],Group,Country,Produced vehicles (2017)[55],Sold vehicles (2018),Sold vehicles (2019)[57]
0,1,Toyota,Japan,10466051,10521134,10741556
1,2,Volkswagen Group,Germany,10382334,10831232,10975352
2,3,General Motors (except SAIC-GM-Wuling)[b],United States,"9,027,658 (6,856,880)",8787233,7724163
3,4,Hyundai,South Korea,7218391,7437209,7189893
4,5,Ford,United States,6386818,5734217,5385972
5,6,Nissan,Japan,5769277,5653743,5176211
6,7,Honda,Japan,5235842,5265892,5323319
7,8,Fiat-Chrysler (now part of Stellantis),Italy / United States,4600847,4841366,4612673
8,9,Renault,France,4153589,3883987,3749815
9,10,PSA Group (now part of Stellantis),France,3649742,4126349,3479152


In [ ]:
total_prod = df_totals.sort_values(by='Year').iloc[-1]['Production']
total_prod

85016728

In [ ]:
#df_by_man['share'] = df_by_man['Vehicles'].apply(lambda x : float(x)/ total_prod)
df_by_man['share'] = df_by_man['Sold vehicles (2019)[57]'].apply(lambda x : float(x)/ total_prod)
market_share = df_by_man.set_index('Group')['share'][['Toyota', 'Volkswagen Group']]
market_share

,share
Group,
Toyota,0.126346
Volkswagen Group,0.129096


In [ ]:
# Leer los datos desde Wikipedia
rev = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_oil_and_gas_companies_by_revenue", header=1)[0]

# Seleccionar las columnas que nos interesan y renombrarlas
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']

# Limpiar la columna 'Revenue' usando una expresión regular para extraer los números
rev['Revenue'] = rev['Revenue'].apply(lambda x: re.findall(r'\d+\.\d+|\d+', str(x))[0] if re.findall(r'\d+\.\d+|\d+', str(x)) else None)

# Convertir los valores limpios a tipo float
rev['Revenue'] = rev['Revenue'].astype(float)

# Mostrar el DataFrame limpio
print(rev)


                Company  Revenue
0        Sonangol Group     17.6
1                   YPF     15.3
2             APA Group      1.9
3         Origin Energy     10.6
4                Santos      3.1
..                  ...      ...
158       Valero Energy     93.0
159  Williams Companies      8.0
160       Uzbekneftegaz      NaN
161               PDVSA      NaN
162        Petrovietnam     21.3

[163 rows x 2 columns]


In [ ]:
total = rev['Revenue'].sum()
total

4930.2

In [ ]:
rev['share'] = rev['Revenue'] / total
rev

,Company,Revenue,share
0,Sonangol Group,17.6,0.003570
1,YPF,15.3,0.003103
2,APA Group,1.9,0.000385
3,Origin Energy,10.6,0.002150
4,Santos,3.1,0.000629
...,...,...,...
158,Valero Energy,93.0,0.018863
159,Williams Companies,8.0,0.001623
160,Uzbekneftegaz,NaN,NaN
161,PDVSA,NaN,NaN


In [ ]:
rev = rev[rev['Company'].isin(['Exxon Mobil', 'Sinopec', 'China National Petroleum Corporation', 'Royal Dutch Shell'])].copy()
rev

,Company,Revenue,share,name
42,China National Petroleum Corporation,346.3,0.070241,China National Petroleum


In [ ]:
# do fuzzy search
split_names = {i: set(i.split(' ')) for i in df['Company']}

def fuzzy(word):
    return fuzzy_match(word, split_names)

rev['name'] = rev['Company'].apply(fuzzy)
rev

,Company,Revenue,share,name
42,China National Petroleum Corporation,346.3,0.070241,China National Petroleum


In [ ]:
ms2 = df.merge(rev[['share', 'name']], left_on='Company', right_on='name')

Ahora queremos juntar todo esto y obtener sólo la empresa y la participación de mercado.

In [ ]:
ms = market_share.reset_index()[['Group','share']]
ms.columns = ['Company', 'share']
pd.concat([ms, ms2[['Company', 'share']]])

,Company,share
0,Toyota,0.126346
1,Volkswagen Group,0.129096
0,China National Petroleum,0.070241
